# Free-range artisanal kerchunk

Let's build a kerchunk reference file _by hand_ for the TIFF we read in the "Reading COGs the hard way" exercise, and see if we can't get that to work with `virtualizarr` to read the tile we read last time.

In [ ]:
import json
import math

from pathlib import Path

import fsspec
import xarray

In [ ]:
import aiohttp
from aiohttp.typedefs import LooseHeaders, StrOrURL

class LoggingClientSession(aiohttp.ClientSession):
    def get(self, url: StrOrURL, *args, headers: LooseHeaders | None = None, **kwargs):
        range_ = ' with no byte range specified'
        if headers and headers.get('Range'):
            range_ = f" with byte range '{headers.get('Range')}'"
        print(f'HTTP Client getting {url}{range_}')
        return super().get(url, *args, headers=headers, **kwargs)


async def get_client(**kwargs):
    return LoggingClientSession(**kwargs)

In [ ]:
tiff_conf = {
    'bucket': 'e84-earth-search-sentinel-data',
    'key': 'sentinel-2-c1-l2a/10/T/FR/2023/12/S2B_T10TFR_20231223T190950_L2A/B04.tif',
    'href': 'https://e84-earth-search-sentinel-data.s3.us-west-2.amazonaws.com/sentinel-2-c1-l2a/10/T/FR/2023/12/S2B_T10TFR_20231223T190950_L2A/B04.tif',
    'size': {
        'rows': 10980,
        'cols': 10980,
    },
    'dtype': '<f4',
    'compression': {
        'id': 'zlib',
    },
    'nodata': 0,
    'scale': 0.0001,
    'offset': -0.1,
    'tiles': {
        'size': {
            'rows': 1024,
            'cols': 1024,
        },
        'offsets': (
            55962680,
            57411167,
            58810332,
            60222446,
            61651003,
            63054996,
            64463518,
            66025043,
            67523672,
            68987825,
            70439668,
            71480485,
            72831139,
            74191906,
            75556803,
            76922917,
            78346396,
            79767466,
            81177106,
            82626646,
            84045343,
            85436959,
            86443457,
            87744763,
            89128625,
            90516041,
            91896145,
            93323921,
            94699513,
            96054131,
            97398784,
            98768288,
            100117176,
            101099165,
            102475360,
            103914015,
            105337327,
            106767167,
            108155055,
            109496290,
            110853634,
            112244713,
            113588526,
            114895196,
            115881834,
            117297939,
            118737048,
            120185270,
            121620456,
            123031867,
            124402930,
            125797422,
            127166585,
            128512496,
            129847193,
            130813023,
            132267746,
            133721219,
            135168258,
            136590051,
            138004961,
            139387400,
            140792216,
            142173401,
            143531515,
            144883653,
            145909014,
            147512228,
            148965842,
            150393926,
            151820955,
            153228408,
            154629691,
            156014882,
            157380367,
            158731976,
            160114338,
            161151032,
            162782880,
            164279805,
            165702744,
            167136236,
            168547793,
            169940007,
            171321112,
            172637466,
            174003270,
            175392224,
            176416464,
            178052532,
            179640701,
            181092961,
            182534128,
            183924802,
            185311293,
            186651878,
            187954039,
            189327902,
            190698439,
            191714854,
            193331795,
            194939195,
            196415899,
            197819251,
            199217398,
            200625142,
            201991437,
            203345078,
            204717082,
            206137392,
            207171477,
            208347309,
            209482357,
            210584255,
            211625167,
            212683632,
            213739512,
            214787222,
            215844136,
            216879023,
            217929856,
        ),
        'lengths': (
            1448479,
            1399157,
            1412106,
            1428549,
            1403985,
            1408514,
            1561517,
            1498621,
            1464145,
            1451835,
            1040809,
            1350646,
            1360759,
            1364889,
            1366106,
            1423471,
            1421062,
            1409632,
            1449532,
            1418689,
            1391608,
            1006490,
            1301298,
            1383854,
            1387408,
            1380096,
            1427768,
            1375584,
            1354610,
            1344645,
            1369496,
            1348880,
            981981,
            1376187,
            1438647,
            1423304,
            1429832,
            1387880,
            1341227,
            1357336,
            1391071,
            1343805,
            1306662,
            986630,
            1416097,
            1439101,
            1448214,
            1435178,
            1411403,
            1371055,
            1394484,
            1369155,
            1345903,
            1334689,
            965822,
            1454715,
            1453465,
            1447031,
            1421785,
            1414902,
            1382431,
            1404808,
            1381177,
            1358106,
            1352130,
            1025353,
            1603206,
            1453606,
            1428076,
            1427021,
            1407445,
            1401275,
            1385183,
            1365477,
            1351601,
            1382354,
            1036686,
            1631840,
            1496917,
            1422931,
            1433484,
            1411549,
            1392206,
            1381097,
            1316346,
            1365796,
            1388946,
            1024232,
            1636060,
            1588161,
            1452252,
            1441159,
            1390666,
            1386483,
            1340577,
            1302153,
            1373855,
            1370529,
            1016407,
            1616933,
            1607392,
            1476696,
            1403344,
            1398139,
            1407736,
            1366287,
            1353633,
            1371996,
            1420302,
            1034077,
            1175824,
            1135040,
            1101890,
            1040904,
            1058457,
            1055872,
            1047702,
            1056906,
            1034879,
            1050825,
            763422,
        ),
    }
}

In [ ]:
kerchunking = {
    '.zgroup': {'zarr_format': 2},
    'red/.zattrs': {'_ARRAY_DIMENSIONS': ['Y', 'X']},
    'red/.zarray': {
        'chunks': [
            tiff_conf['tiles']['size']['rows'],
            tiff_conf['tiles']['size']['cols'],
        ],
        'compressor': tiff_conf['compression'],
        'dtype': tiff_conf['dtype'],
        'fill_value': tiff_conf['nodata'],
        'filters': [
            {
                'id': 'fixedscaleoffset',
                'offset': tiff_conf['offset'],
                'scale': 1/tiff_conf['scale'],
                'dtype': tiff_conf['dtype'],
                'astype': '<u2',
            },
            {
                'id': 'delta',
                'dtype': '<u2',
                'astype': '<u2',
            },
        ],
        'order': 'C',
        'shape': [
            tiff_conf['size']['rows'],
            tiff_conf['size']['cols'],
        ],
        'zarr_format': 2,
    },
}

for tile_row in range(math.ceil(tiff_conf['size']['rows'] / tiff_conf['tiles']['size']['rows'])):
    for tile_col in range(math.ceil(tiff_conf['size']['cols'] / tiff_conf['tiles']['size']['cols'])):
        tile_index = (math.ceil(tiff_conf['size']['cols'] / tiff_conf['tiles']['size']['cols']) * tile_row) + tile_col
        kerchunking[f'red/{tile_row}.{tile_col}'] = [
            tiff_conf['href'],
            #f"{tiff_conf['bucket']}/{tiff_conf['key']}",
            tiff_conf['tiles']['offsets'][tile_index],
            tiff_conf['tiles']['lengths'][tile_index],
        ]

kerchunk_json = json.dumps(kerchunking)

print(kerchunk_json)

Find the array for the tile we red in the COG exercise. Does the offset and length match what we previously found?

In [ ]:
json_file = Path('./kerchunk.json')
json_file.write_text(kerchunk_json)

In [ ]:
dataset = xarray.open_dataset(
    'reference://',
    engine='zarr',
    backend_kwargs={
        'consolidated': False,
        'storage_options': {
            'fo': str(json_file),
            'remote_protocol': 'https',
            'remote_options': {
                'get_client': get_client,
            }
        },
    },
)
dataset

In [ ]:
dataset.red

In [ ]:
dataset.chunks

In [ ]:
dataset.red


In [ ]:
tile_of_interest = dataset.red[1024*7:1024*8,0:1024]
tile_of_interest

In [ ]:
tile_of_interest.values

In [ ]:
two_tiles = dataset.red[1024*7:1024*8,0:1024*2]
two_tiles

In [ ]:
two_tiles.values